In [ ]:
#relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import itertools
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import scipy.stats as stats
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

### The stocks analysed within this project is
#### Goldman Sachs and Citibank - Investment Banks
#### Johnson and Johnson - Pharmaceutical organisation
#### Google and Apple - Big Technology organisations

### Data Cleaning and Transformation

In [ ]:
#testing it out - fetching historical data
#Citibank
citi_df = yf.download("C", start="2015-01-01", end="2025-01-01")
citi_df.head()

In [ ]:
citi_df.describe()

In [ ]:
#Goldman Sachs
goldman_df = yf.download("GS", start="2015-01-01", end="2025-01-01")
goldman_df.head()

In [ ]:
goldman_df.describe()

In [ ]:
#Johnson and Johnson
johnson_df = yf.download("JNJ", start="2015-01-01", end="2025-01-01")
johnson_df.head()

In [ ]:
johnson_df.describe()

In [ ]:
google_df = yf.download("GOOG", start="2015-01-01", end="2025-01-01")
google_df.head()

In [ ]:
google_df.describe()

In [ ]:
apple_df = yf.download("AAPL", start="2015-01-01", end="2025-01-01")
apple_df.head()

In [ ]:
apple_df.describe()

In [ ]:
#checking for null values for each dataframe

null_citi = citi_df.isnull().sum()
null_goldman = goldman_df.isnull().sum()
null_johnson = johnson_df.isnull().sum()
null_google = google_df.isnull().sum()
null_apple = apple_df.isnull().sum()

print(null_citi)
print(null_goldman)
print(null_johnson)
print(null_google)
print(null_apple)

In [ ]:
#checking the data type for each column

citi_type = citi_df.dtypes
goldman_type = goldman_df.dtypes
johnson_type = johnson_df.dtypes
google_type = google_df.dtypes
apple_type = apple_df.dtypes

print(citi_type)
print(goldman_type)
print(johnson_type)
print(google_type)
print(apple_type)

### Initial observations

##### There are no null values within any of the dataframes examined. The data types of each column is consistent through all of the dataframes.

#### Explanation of each column, when reviewing a stock price

##### Close - the price of a stock when the markets close (the target variable)
##### High - the price of a stock at its highest for that day
##### Low - the price of a stock at its lowest for that day
##### Open - the price of a stock when the markets close
##### Volume - the number of shares traded, in relation to the stock

### Functions to be used within this project

In [ ]:
def plot_time_series (df, column_name, title, xlabel='Time Period', ylabel='Closing Price', color='red'):

  """
  This function plots a time series from the specified dataframe.

  Parameters:
  df: The specified dataframe
  column_name: The name of the column to plot
  title: The title of the plot
  xlabel: The label for the x-axis
  ylabel: The label for the y-axis
  color: The color of the plot line

  Returns:
  A line plot of the time series
  """

  plt.figure(figsize=(16, 8))
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.title(title)
  plt.plot(df.index, df[column_name], color=color)
  plt.show()

In [ ]:
def add_sma_column (df, column_name, window):
  """
  This function creates a new dataframe based on the input dataframe and adds a Simple Moving Average (SMA) column.

  Parameters:
  df: The input dataframe
  column_name: The name of the column to calculate the SMA on
  window: The rolling window size for the SMA calculation

  Returns:
  A new Dataframe with the original data and the added SMA column
  """

  new_df = df.copy()
  new_df[f'SMA{window}'] = new_df[column_name].rolling(window).mean()
  return new_df

In [ ]:
def plot_close_sma(df, close_column, sma_column, window, title, xlabel='Time Period', ylabel='Closing Price', close_color='crimson', sma_color='yellowgreen'):
  """
  This function plots the closing price and the Simple Moving Average (SMA) of the specified dataframe, to compare the two.

  Parameters:
  df: The specified dataframe
  close_column: The name of the column to plot the closing price
  sma_column: The name of the column to plot the SMA
  window: The rolling window size for the SMA calculation
  title: The title of the plot
  xlabel: The label for the x-axis
  ylabel: The label for the y-axis
  close_color: The color of the closing price line
  sma_color: The color of the SMA line

  Returns:
  A line plot side by side of the closing price and the SMA
  """
  plt.figure(figsize=(16, 8))
  df[close_column].plot(label=f'{close_column}', color=close_color)
  df[sma_column].plot(label=f'SMA{window}', color=sma_color)
  plt.title(title)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.legend()
  plt.show()


In [ ]:
def decompose(df, column_name, period=None):
    """
    This function decomposes a time series, by performing multiplicative seasonal decomposition
    on the specified dataframe, and subsequent column.

    Parameters:
        df: The specified data frame.
        column_name: The specified column.
        period: The period of the time series/seasonality.

    Returns:
        The trend, seasonal and residual components of the time series as a plot for each component.
    """

    if period is None and df.index.freq is None:
      print("Warning: No period specified and index has no frequency. Setting period to 5 as a default")
      print("This plot showcases the multiplicative decompose of the specified dataframe")
      period = 5 #default period for daily data
      result = seasonal_decompose(df[column_name], model='multiplicative', period=period)

    elif period is not None:
      result = seasonal_decompose(df[column_name], model='multiplicative', period=period)

    else:
      result = seasonal_decompose(df[column_name], model='multiplicative')


    plt.rcParams.update({'figure.figsize': (20, 10)})
    result.plot()

    return result


In [ ]:
#performing the adf test - this function to be modular - used in other cells in the notebook

def adf_test(df, column_name):
  """
  This function performs the Adfuller test on the specified datatframe.

  Parameters:
  df: The specified data frame.
  column_name: The specified column.

  Returns:
   Various print statements which detail the ADF statistic, P-value and critical values.

  """
  data = df[column_name].dropna()
  X = data.values
  results = adfuller(X)
  print('ADF Statistic: %f' % results[0])
  print('P-value: %f' % results[1])
  print('Critical Values:')
  for key, value in results[4].items():
    print('\t%s: %.3f' % (key, value))

  return results


In [ ]:
def plot_acf_pacf(df, column_name, lags=40, title_suffix=""):
  """
  This function plots the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF)
  for the 'Close column in the specified dataframe.

  Parameters:
  df: The dataframe containing the time series data.
  column_name: The name of the column to plot ACF and PACF for.
  lags: The number of lags to display in the plots - set at lags = 40 to make plot readable.
  title_suffix: A string to append to the plot titles (e.g., stock ticker).

  Returns:
  Displays ACF and PACF plots.
  """
  fig, axes = plt.subplots(1, 2, figsize=(16, 6))
  plot_acf(df[column_name].dropna(), lags=lags, ax=axes[0])
  axes[0].set_title(f'Autocorrelation Function (ACF) - {column_name}{title_suffix}')
  plot_pacf(df[column_name].dropna(), lags=lags, ax=axes[1])
  axes[1].set_title(f'Partial Autocorrelation Function (PACF) - {column_name}{title_suffix}')
  plt.show()

In [ ]:
def calculate_and_plot_log_returns(df, column_name, title_suffix=""):
  """
  This function calculates the log returns for the 'Close' column within the specified dataframe,
  plots the log returns over time, and displays a histogram and Q-Q plot
  to visualise the distribution of the log returns.

  Parameters:
  df: The dataframe containing the time series data.
  column_name: The name of the column to calculate log returns for (usually 'Close').
  title_suffix: A string to append to the plot titles - stock name.

  Returns:
  Displays a plot of log returns, a histogram, and a Q-Q plot.
  """

  #calculates the log returns
  log_returns = np.log(df[column_name] / df[column_name].shift(1)).dropna()

  #plots log returns over time
  plt.figure(figsize=(16, 6))
  log_returns.plot()
  plt.title(f'Log Returns - {column_name}{title_suffix}')
  plt.xlabel('Time Period')
  plt.ylabel('Log Return')
  plt.show()

  #plots histogram of log returns
  plt.figure(figsize=(10, 6))
  sns.histplot(log_returns, kde=True, bins=50)
  plt.title(f'Distribution of Log Returns - {column_name}{title_suffix}')
  plt.xlabel('Log Return')
  plt.ylabel('Frequency')
  plt.show()

In [ ]:
def first_order_diff(df, price_type, ticker):
  """
  This function performs a first-order differencing on the specified dataframe.

  Parameters:
  df: The specified dataframe
  column_name: The name of the column to perform first-order differencing on

  Returns:
  A new dataframe with the first-order differenced column
  """
  df = df.copy()
  col = (price_type, ticker)
  diff_col_name = f"{price_type}_{ticker}_diff"
  df[diff_col_name] = df[col].diff()
  display(df[[col]].join(df[[diff_col_name]]).tail())
  return df

## Exploratory Data Analysis

In [ ]:
stocks = {
    "AAPL": apple_df,
    "GOOG": google_df,
    "GS": goldman_df,
    "JNJ": johnson_df,
    "C": citi_df
}

#computing the log returns and concatenate into a single df
returns_list = []
for ticker, df in stocks.items():
    df = df.copy()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df = df[['Log_Returns']].dropna()
    df.rename(columns={'Log_Returns': ticker}, inplace=True)
    returns_list.append(df)

returns_df = pd.concat(returns_list, axis=1)

#plots
plt.figure(figsize=(10, 6))
returns_df.boxplot()
plt.title("Boxplot of Daily Log Returns for All Stocks")
plt.ylabel("Log Returns")
plt.grid(True, alpha=0.4)
plt.show()

In [ ]:
outlier_counts = {}

for ticker in returns_df.columns:
    series = returns_df[ticker].dropna()
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = series[(series < lower) | (series > upper)]
    outlier_counts[ticker] = len(outliers)

print(outlier_counts)

### Apple

In [ ]:
plot_time_series(apple_df, 'Close', 'Apple Stock Price', color='blueviolet')

In [ ]:
apple_sma = add_sma_column(apple_df, 'Close', 30)

In [ ]:
apple_sma.head()

In [ ]:
plot_close_sma(apple_sma, 'Close', 'SMA30', 30, 'Apple Stock Price with 30 Day SMA')

In [ ]:
result = decompose(apple_df, 'Close')

In [ ]:
results = adf_test(apple_df, 'Close')

In [ ]:
plot_acf_pacf(apple_df, 'Close', title_suffix=' - Apple')

In [ ]:
calculate_and_plot_log_returns(apple_df, 'Close', title_suffix=' - Apple')

### Citi

In [ ]:
plot_time_series(citi_df, 'Close', 'Citi Stock Price', color='lightcoral')

In [ ]:
citi_sma = add_sma_column(citi_df, 'Close', 30)

In [ ]:
citi_sma.head()

In [ ]:
plot_close_sma(citi_sma, 'Close', 'SMA30', 30, 'Citi Stock Price with 30 Day SMA')

In [ ]:
result = decompose(citi_df, 'Close')

In [ ]:
results = adf_test(citi_df, 'Close')

In [ ]:
plot_acf_pacf(citi_df, 'Close', title_suffix=' - Citi')

In [ ]:
calculate_and_plot_log_returns(citi_df, 'Close', title_suffix=' - Citi')

### Goldman Sachs

In [ ]:
plot_time_series(goldman_df, 'Close', 'Goldman Sachs Stock Price', color='goldenrod')

In [ ]:
goldman_sma = add_sma_column(goldman_df, 'Close', 30)

In [ ]:
goldman_sma.head()

In [ ]:
plot_close_sma(goldman_sma, 'Close', 'SMA30', 30, 'Goldman Sachs Stock Price with 30 Day SMA')

In [ ]:
result = decompose(goldman_df, 'Close')

In [ ]:
results = adf_test(goldman_df, 'Close')

In [ ]:
plot_acf_pacf(goldman_df, 'Close', title_suffix=' - GS')

In [ ]:
calculate_and_plot_log_returns(goldman_df, 'Close', title_suffix=' - GS')

### Google

In [ ]:
plot_time_series(google_df, 'Close', 'Google Stock Price', color='cornflowerblue')

In [ ]:
google_sma = add_sma_column(google_df, 'Close', 30)

In [ ]:
google_sma.head()

In [ ]:
plot_close_sma(google_sma, 'Close', 'SMA30', 30, 'Google Stock Price with 30 Day SMA')

In [ ]:
result = decompose(google_df, 'Close')

In [ ]:
results = adf_test(google_df, 'Close')

In [ ]:
plot_acf_pacf(google_df, 'Close', title_suffix=' - Google')

In [ ]:
calculate_and_plot_log_returns(google_df, 'Close', title_suffix=' - Google')

### Johnson and Johnson

In [ ]:
plot_time_series(johnson_df, 'Close', 'JNJ Stock Price', color='mediumseagreen')

In [ ]:
johnson_sma = add_sma_column(johnson_df, 'Close', 30)

In [ ]:
johnson_sma.head()

In [ ]:
plot_close_sma(johnson_sma, 'Close', 'SMA30', 30, 'JNJ Stock Price with 30 Day SMA')

In [ ]:
result = decompose(johnson_df, 'Close')

In [ ]:
results = adf_test(johnson_df, 'Close')

In [ ]:
plot_acf_pacf(johnson_df, 'Close', title_suffix=' - JNJ')

In [ ]:
calculate_and_plot_log_returns(johnson_df, 'Close', title_suffix=' - JNJ')

## Model Building

### Traditional Time Series Models

#### ARIMA (Auto-regressive moving average)

###### Notes: All stocks in this project are non-stationary, due to the nature of the time series - trends upwards and downwards so differencing is performed on all the stocks - update after differencing - all time series are now stationary, ready for model building

##### Apple

In [ ]:
#apple
apple_df_new = first_order_diff(apple_df, 'Close', 'AAPL')

In [ ]:
#reviewing effects
results = adf_test(apple_df_new, 'Close_AAPL_diff')

In [ ]:
apple_df_new.head()

In [ ]:
close_prices = apple_df_new['Close']

#define the p, d, q ranges
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = ARIMA(close_prices, order=order)
        results = model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except:
        continue

print(f"Best ARIMA order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#define train and test set in the time series, then build the ARIMA model, alongside its summary

train = apple_df_new['Close'][:-30]
test = apple_df_new['Close'][-30:]

apple_model = ARIMA(train, order=(0, 1, 0))
apple_model_fit = apple_model.fit()
print(apple_model_fit.summary())

In [ ]:
#forecasting values up to 30 steps, and calculating error metrics

forecast = apple_model_fit.forecast(steps=30)
print(forecast)

rmse = np.sqrt(mean_squared_error(test, forecast))
mae = mean_absolute_error(test, forecast)
mape = mean_absolute_percentage_error(test, forecast)

print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2%}")

In [ ]:
#plot actual vs forecast values - apple
plt.figure(figsize=(10, 5))
plt.plot(test.index, test, label='Actual', color='black')
plt.plot(test.index, forecast, label='Forecast', color='red')
plt.title("ARIMA Forecast vs Actual - Apple")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

##### Citi

In [ ]:
#citi
citi_df_new = first_order_diff(citi_df, 'Close', 'C')

In [ ]:
#reviewing effects
results = adf_test(citi_df_new, 'Close_C_diff')

In [ ]:
close_prices = citi_df_new['Close']

#define the p, d, q ranges
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = ARIMA(close_prices, order=order)
        results = model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except:
        continue

print(f"Best ARIMA order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#define train and test set in the time series, then build the ARIMA model, alongside its summary

train = citi_df_new['Close'][:-30]
test = citi_df_new['Close'][-30:]

citi_model = ARIMA(train, order=(0, 1, 2))
citi_model_fit = citi_model.fit()
print(citi_model_fit.summary())

In [ ]:
#forecasting values up to 30 steps, and calculating error metrics

forecast = citi_model_fit.forecast(steps=30)
print(forecast)

rmse = np.sqrt(mean_squared_error(test, forecast))
mae = mean_absolute_error(test, forecast)
mape = mean_absolute_percentage_error(test, forecast)

print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2%}")

In [ ]:
#plot actual vs forecast values - citi
plt.figure(figsize=(10, 5))
plt.plot(test.index, test, label='Actual', color='black')
plt.plot(test.index, forecast, label='Forecast', color='lightcoral')
plt.title("ARIMA Forecast vs Actual - Citi")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

##### Goldman Sachs

In [ ]:
#goldman sachs
goldman_df_new = first_order_diff(goldman_df, 'Close', 'GS')

In [ ]:
#reviewing effects
results = adf_test(goldman_df_new, 'Close_GS_diff')

In [ ]:
close_prices = goldman_df_new['Close']

#function which chooses the best ARIMA order for time series

p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = ARIMA(close_prices, order=order)
        results = model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except:
        continue

print(f"Best ARIMA order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#define train and test set
train = goldman_df_new['Close'][:-30]
test = goldman_df_new['Close'][-30:]

#build model
goldman_model = ARIMA(train, order=(1, 1, 2))
goldman_model_fit = goldman_model.fit()
print(goldman_model_fit.summary())

In [ ]:
# Forecasting the next 30 steps
forecast = goldman_model_fit.forecast(steps=30)
print(forecast)

rmse = np.sqrt(mean_squared_error(test, forecast))
mae = mean_absolute_error(test, forecast)
mape = mean_absolute_percentage_error(test, forecast)

print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2%}")

In [ ]:
#plot actual vs forecast values - goldman sachs
plt.figure(figsize=(10, 5))
plt.plot(test.index, test, label='Actual', color='black')
plt.plot(test.index, forecast, label='Forecast', color='goldenrod')
plt.title("ARIMA Forecast vs Actual - Goldman Sachs")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

##### Google

In [ ]:
#google
google_df_new = first_order_diff(google_df, 'Close', 'GOOG')

In [ ]:
#reviewing effects
results = adf_test(google_df_new, 'Close_GOOG_diff')

In [ ]:
close_prices = google_df_new['Close']

#function which chooses the best ARIMA order for time series
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = ARIMA(close_prices, order=order)
        results = model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except:
        continue

print(f"Best ARIMA order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#define train and test set in the time series, then build the ARIMA model, alongside its summary

train = google_df_new['Close'][:-30]
test = google_df_new['Close'][-30:]

google_model = ARIMA(train, order=(2, 1, 2))
google_model_fit = google_model.fit()
print(google_model_fit.summary())

In [ ]:
#forecasting values up to 30 steps, and calculating error metrics

forecast = google_model_fit.forecast(steps=30)
print(forecast)

rmse = np.sqrt(mean_squared_error(test, forecast))
mae = mean_absolute_error(test, forecast)
mape = mean_absolute_percentage_error(test, forecast)

print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2%}")

In [ ]:
#plot actual vs forecast values - google
plt.figure(figsize=(10, 5))
plt.plot(test.index, test, label='Actual', color='black')
plt.plot(test.index, forecast, label='Forecast', color='cornflowerblue')
plt.title("ARIMA Forecast vs Actual - Google")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

##### Johnson and Johnson

In [ ]:
#johnson-johnson
jnj_df_new = first_order_diff(johnson_df, 'Close', 'JNJ')

In [ ]:
#reviewing effects
results = adf_test(jnj_df_new, 'Close_JNJ_diff')

In [ ]:
close_prices = jnj_df_new['Close']

#function which chooses the best ARIMA order for time series
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = ARIMA(close_prices, order=order)
        results = model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except:
        continue

print(f"Best ARIMA order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#define train and test set in the time series, then build the ARIMA model, alongside its summary

train = jnj_df_new['Close'][:-30]
test = jnj_df_new['Close'][-30:]

jnj_model = ARIMA(train, order=(2, 1, 2))
jnj_model_fit = jnj_model.fit()
print(jnj_model_fit.summary())

In [ ]:
#forecasting values up to 30 steps, and calculating error metrics

forecast = jnj_model_fit.forecast(steps=30)
print(forecast)

rmse = np.sqrt(mean_squared_error(test, forecast))
mae = mean_absolute_error(test, forecast)
mape = mean_absolute_percentage_error(test, forecast)

print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}, MAPE: {mape:.2%}")

In [ ]:
#plot actual vs forecast values - johnson and johnson
plt.figure(figsize=(10, 5))
plt.plot(test.index, test, label='Actual', color='black')
plt.plot(test.index, forecast, label='Forecast', color='mediumseagreen')
plt.title("ARIMA Forecast vs Actual - Johnson and Johnson")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### SARIMAX (Seasonal Autoregressive Integrated Moving Average with Exogenous Regressors)

##### Goldman Sachs

In [ ]:
#downloading sp 500 to use as an exogenous variable

sp_500 = yf.download('^GSPC', start='2015-01-01', end='2025-01-01')
sp_500.head()

In [ ]:
gs_sarima = pd.concat([goldman_df["Close"], sp_500["Close"]], axis=1)
gs_sarima.columns = ["GS_Close", "SP500_Close"]
gs_sarima = gs_sarima.dropna()

In [ ]:
gs_sarima.head()

In [ ]:
#performing stationary, differencing

gs_sarima["GS_Returns"] = np.log(gs_sarima["GS_Close"] / gs_sarima["GS_Close"].shift(1))
gs_sarima["SP500_Returns"] = np.log(gs_sarima["SP500_Close"] / gs_sarima["SP500_Close"].shift(1))
gs_sarima = gs_sarima.dropna()

In [ ]:
#defining X and y data

y = gs_sarima["GS_Returns"].reset_index(drop=True)
X = gs_sarima[["SP500_Returns"]].reset_index(drop=True)

In [ ]:
#train and test split

train_size = int(len(y) * 0.8)
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]

In [ ]:
p = q = range(0, 3)
d = [0]  # returns are already stationary
pdq = list(itertools.product(p, d, q))

best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = SARIMAX(
            y_train,
            order=order,
            seasonal_order=(0, 0, 0, 0),  # no seasonality for stock returns
            exog=X_train,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        results = model.fit(disp=False)

        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except Exception as e:
        continue


print(f"Best SARIMAX order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#to forecast returns
forecast = best_model.get_forecast(steps=len(y_test), exog=X_test)
forecast_mean = forecast.predicted_mean

#metrics on returns
rmse_ret = np.sqrt(mean_squared_error(y_test, forecast_mean))
mae_ret = mean_absolute_error(y_test, forecast_mean)

print("Metrics on Returns")
print(f"RMSE: {rmse_ret:.6f}, MAE: {mae_ret:.6f}")

#to reconstruct prices
goldman_prices = gs_sarima["GS_Close"].reset_index(drop=True)

# actual test prices
actual_prices = goldman_prices[train_size:]

# forecasted returns -> cumulative sum -> exponentiate -> multiply by last train price
last_train_price = goldman_prices[train_size - 1]
forecast_prices = last_train_price * np.exp(forecast_mean.cumsum())

# Metrics on prices
rmse_price = np.sqrt(mean_squared_error(actual_prices, forecast_prices))
mae_price = mean_absolute_error(actual_prices, forecast_prices)
mape_price = mean_absolute_percentage_error(actual_prices, forecast_prices)

print("Metrics on Prices")
print(f"RMSE: {rmse_price:.2f}, MAE: {mae_price:.2f}, MAPE: {mape_price:.2%}")

##### Citi

In [ ]:
citi_sarima = pd.concat([citi_df["Close"], sp_500["Close"]], axis=1)
citi_sarima.columns = ["Citi_Close", "SP500_Close"]
citi_sarima = citi_sarima.dropna()

In [ ]:
citi_sarima.head()

In [ ]:
#performing stationary, differencing

citi_sarima["Citi_Returns"] = np.log(citi_sarima["Citi_Close"] / citi_sarima["Citi_Close"].shift(1))
citi_sarima["SP500_Returns"] = np.log(citi_sarima["SP500_Close"] / citi_sarima["SP500_Close"].shift(1))
citi_sarima = citi_sarima.dropna()

#defining X and y data

y = citi_sarima["Citi_Returns"].reset_index(drop=True)
X = citi_sarima[["SP500_Returns"]].reset_index(drop=True)


#train and test split

train_size = int(len(y) * 0.8)
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]

In [ ]:
p = q = range(0, 3)
d = [0]  # returns are already stationary
pdq = list(itertools.product(p, d, q))
best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = SARIMAX(
            y_train,
            order=order,
            seasonal_order=(0, 0, 0, 0),  # no seasonality for stock returns
            exog=X_train,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        results = model.fit(disp=False)

        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except Exception as e:
        continue


print(f"Best SARIMAX order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#forecast returns
forecast = best_model.get_forecast(steps=len(y_test), exog=X_test)
forecast_mean = forecast.predicted_mean

#metrics on returns
rmse_ret = np.sqrt(mean_squared_error(y_test, forecast_mean))
mae_ret = mean_absolute_error(y_test, forecast_mean)

print("Metrics on Returns")
print(f"RMSE: {rmse_ret:.6f}, MAE: {mae_ret:.6f}")


#reconstruct prices
citi_prices = citi_sarima["Citi_Close"].reset_index(drop=True)

# actual test prices
actual_prices = citi_prices[train_size:]

# forecasted returns -> cumulative sum -> exponentiate -> multiply by last train price
last_train_price = citi_prices[train_size - 1]
forecast_prices = last_train_price * np.exp(forecast_mean.cumsum())

#metrics on prices
rmse_price = np.sqrt(mean_squared_error(actual_prices, forecast_prices))
mae_price = mean_absolute_error(actual_prices, forecast_prices)
mape_price = mean_absolute_percentage_error(actual_prices, forecast_prices)

print("Metrics on Prices")
print(f"RMSE: {rmse_price:.2f}, MAE: {mae_price:.2f}, MAPE: {mape_price:.2%}")

##### Apple

In [ ]:
apple_sarima = pd.concat([apple_df["Close"], sp_500["Close"]], axis=1)
apple_sarima.columns = ["Apple_Close", "SP500_Close"]
apple_sarima = apple_sarima.dropna()

apple_sarima.head()


#performing stationary, differencing

apple_sarima["Apple_Returns"] = np.log(apple_sarima["Apple_Close"] / apple_sarima["Apple_Close"].shift(1))
apple_sarima["SP500_Returns"] = np.log(apple_sarima["SP500_Close"] / apple_sarima["SP500_Close"].shift(1))
apple_sarima = apple_sarima.dropna()

#defining X and y data

y = apple_sarima["Apple_Returns"].reset_index(drop=True)
X = apple_sarima[["SP500_Returns"]].reset_index(drop=True)

#train and test split

train_size = int(len(y) * 0.8)
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]

In [ ]:
p = q = range(0, 3)
d = [0]  # returns are already stationary
pdq = list(itertools.product(p, d, q))
best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = SARIMAX(
            y_train,
            order=order,
            seasonal_order=(0, 0, 0, 0),  # no seasonality for stock returns
            exog=X_train,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        results = model.fit(disp=False)

        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except Exception as e:
        continue


print(f"Best SARIMAX order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#forecast returns
forecast = best_model.get_forecast(steps=len(y_test), exog=X_test)
forecast_mean = forecast.predicted_mean

#metrics on returns
rmse_ret = np.sqrt(mean_squared_error(y_test, forecast_mean))
mae_ret = mean_absolute_error(y_test, forecast_mean)

print("Metrics on Returns")
print(f"RMSE: {rmse_ret:.6f}, MAE: {mae_ret:.6f}")

#reconstruct prices
apple_prices = apple_sarima["Apple_Close"].reset_index(drop=True)

# actual test prices
actual_prices = apple_prices[train_size:]

# forecasted returns -> cumulative sum -> exponentiate -> multiply by last train price
last_train_price = apple_prices[train_size - 1]
forecast_prices = last_train_price * np.exp(forecast_mean.cumsum())

#metrics on prices
rmse_price = np.sqrt(mean_squared_error(actual_prices, forecast_prices))
mae_price = mean_absolute_error(actual_prices, forecast_prices)
mape_price = mean_absolute_percentage_error(actual_prices, forecast_prices)

print("Metrics on Prices")
print(f"RMSE: {rmse_price:.2f}, MAE: {mae_price:.2f}, MAPE: {mape_price:.2%}")

In [ ]:
#last known actual price before test period
last_price = apple_sarima["Apple_Close"].iloc[train_size]

#forecasted returns
forecast_returns = forecast_mean.values

#convert returns to prices
forecast_prices = [last_price]
for r in forecast_returns:
    next_price = forecast_prices[-1] * np.exp(r)
    forecast_prices.append(next_price)

#drop the initial last_price (keep only forecasted steps)
forecast_prices = forecast_prices[1:]

#actual test prices
actual_prices = apple_sarima["Apple_Close"].iloc[train_size:].values

#plot
plt.figure(figsize=(12,6))
plt.plot(actual_prices, label="Actual Price", color="black")
plt.plot(forecast_prices, label="SARIMAX Forecast (Price)", color="red", linestyle="--")
plt.title("SARIMAX Forecast vs Actual (AAPL Closing Price)")
plt.xlabel("Time Index (Test Period)")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

##### Google

In [ ]:
google_sarima = pd.concat([google_df["Close"], sp_500["Close"]], axis=1)
google_sarima.columns = ["Google_Close", "SP500_Close"]
google_sarima = google_sarima.dropna()

google_sarima.head()


#performing stationary, differencing

google_sarima["Google_Returns"] = np.log(google_sarima["Google_Close"] / google_sarima["Google_Close"].shift(1))
google_sarima["SP500_Returns"] = np.log(google_sarima["SP500_Close"] / google_sarima["SP500_Close"].shift(1))
google_sarima = google_sarima.dropna()

#defining X and y data

y = google_sarima["Google_Returns"].reset_index(drop=True)
X = google_sarima[["SP500_Returns"]].reset_index(drop=True)

#train and test split

train_size = int(len(y) * 0.8)
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]

In [ ]:
p = q = range(0, 3)
d = [0]  # returns are already stationary
pdq = list(itertools.product(p, d, q))
best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = SARIMAX(
            y_train,
            order=order,
            seasonal_order=(0, 0, 0, 0),  # no seasonality for stock returns
            exog=X_train,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        results = model.fit(disp=False)

        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except Exception as e:
        continue


print(f"Best SARIMAX order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:
#forecast returns
forecast = best_model.get_forecast(steps=len(y_test), exog=X_test)
forecast_mean = forecast.predicted_mean

#metrics on returns
rmse_ret = np.sqrt(mean_squared_error(y_test, forecast_mean))
mae_ret = mean_absolute_error(y_test, forecast_mean)

print("Metrics on Returns")
print(f"RMSE: {rmse_ret:.6f}, MAE: {mae_ret:.6f}")

#reconstruct prices
google_prices = google_sarima["Google_Close"].reset_index(drop=True)

# actual test prices
actual_prices = google_prices[train_size:]

# forecasted returns -> cumulative sum -> exponentiate -> multiply by last train price
last_train_price = google_prices[train_size - 1]
forecast_prices = last_train_price * np.exp(forecast_mean.cumsum())

#metrics on prices
rmse_price = np.sqrt(mean_squared_error(actual_prices, forecast_prices))
mae_price = mean_absolute_error(actual_prices, forecast_prices)
mape_price = mean_absolute_percentage_error(actual_prices, forecast_prices)

print("Metrics on Prices")
print(f"RMSE: {rmse_price:.2f}, MAE: {mae_price:.2f}, MAPE: {mape_price:.2%}")

##### Johnson and Johnson

In [ ]:
jnj_sarima = pd.concat([johnson_df["Close"], sp_500["Close"]], axis=1)
jnj_sarima.columns = ["Johnson_Close", "SP500_Close"]
jnj_sarima = jnj_sarima.dropna()

jnj_sarima.head()


#performing stationary, differencing

jnj_sarima["Johnson_Returns"] = np.log(jnj_sarima["Johnson_Close"] / jnj_sarima["Johnson_Close"].shift(1))
jnj_sarima["SP500_Returns"] = np.log(jnj_sarima["SP500_Close"] / jnj_sarima["SP500_Close"].shift(1))
jnj_sarima = jnj_sarima.dropna()

#defining X and y data

y = jnj_sarima["Johnson_Returns"].reset_index(drop=True)
X = jnj_sarima[["SP500_Returns"]].reset_index(drop=True)

#train and test split

train_size = int(len(y) * 0.8)
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]

In [ ]:
p = q = range(0, 3)
d = [0]  # returns are already stationary
pdq = list(itertools.product(p, d, q))
best_aic = np.inf
best_order = None
best_model = None

for order in pdq:
    try:
        model = SARIMAX(
            y_train,
            order=order,
            seasonal_order=(0, 0, 0, 0),  # no seasonality for stock returns
            exog=X_train,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        results = model.fit(disp=False)

        if results.aic < best_aic:
            best_aic = results.aic
            best_order = order
            best_model = results
    except Exception as e:
        continue


print(f"Best SARIMAX order: {best_order} with AIC: {best_aic}")
print(best_model.summary())

In [ ]:

forecast = best_model.get_forecast(steps=len(y_test), exog=X_test)
forecast_mean = forecast.predicted_mean


rmse_ret = np.sqrt(mean_squared_error(y_test, forecast_mean))
mae_ret = mean_absolute_error(y_test, forecast_mean)

print("Metrics on Returns")
print(f"RMSE: {rmse_ret:.6f}, MAE: {mae_ret:.6f}")


jnj_prices = jnj_sarima["Johnson_Close"].reset_index(drop=True)

actual_prices = jnj_prices[train_size:]

last_train_price = jnj_prices[train_size - 1]
forecast_prices = last_train_price * np.exp(forecast_mean.cumsum())

rmse_price = np.sqrt(mean_squared_error(actual_prices, forecast_prices))
mae_price = mean_absolute_error(actual_prices, forecast_prices)
mape_price = mean_absolute_percentage_error(actual_prices, forecast_prices)

print("Metrics on Prices")
print(f"RMSE: {rmse_price:.2f}, MAE: {mae_price:.2f}, MAPE: {mape_price:.2%}")

### Deep Learning Methods

#### LSTM

##### Apple

In [ ]:
data = apple_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])  #predict next 30
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)

#reshape for LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))

#build model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)  # directly predict 30 steps
])

model.compile(optimizer='adam', loss='mse')

#train
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)

#forecast
last_seq = scaled_data[-60:]  # last 60 days
last_seq = last_seq.reshape((1, 60, 1))
pred = model.predict(last_seq)
pred = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()

print("Next 30-day forecast:", pred)

In [ ]:
y_true = data[-30:].flatten()  # these are already in real price units

for i in range(5):
    print(f"Day {i+1}: Pred={pred[i]:.2f}, Actual={y_true[i]:.2f}")

In [ ]:
print("Hello!")

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(range(len(y_true)), y_true, label="Actual Price", color="black")
plt.plot(range(len(pred)), pred, label="LSTM Forecast (Univariate)", color="red", linestyle="--")

plt.title("LSTM Forecast vs Actual (AAPL Closing Price)")
plt.xlabel("Days Ahead (Test Horizon)")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

In [ ]:
#RMSE
mse = mean_squared_error(y_true, pred)
rmse = (mse)**(1/2)
#MAE
mae = mean_absolute_error(y_true, pred)
#MAPE
mape = np.mean(np.abs((y_true - pred) / y_true)) * 100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
#close and exog

apple_lstm = pd.concat([apple_df["Close"], sp_500["Close"]], axis=1)
apple_lstm.columns = ["Apple_Close", "SP500_Close"]
apple_lstm = apple_lstm.dropna()

apple_lstm.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(apple_lstm)

def create_sequences_multivariate(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        #input includes both features
        X.append(data[i-past_steps:i, :])
        #target is only the stock Close (column 0)
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences_multivariate(scaled_data, 60, 30)

model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)  # predict next 30 days
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)

#last 60 days of features (stock + sp500)
last_seq = scaled_data[-60:]
last_seq = last_seq.reshape((1, 60, 2))

y_pred = model.predict(last_seq)
y_pred = scaler.inverse_transform(
    np.concatenate([y_pred.reshape(-1,1), np.zeros((30,1))], axis=1)
)[:,0]  # keep only stock column

# actual values for comparison
y_true = apple_lstm['Apple_Close'].values[-30:]

for i in range(5):
    print(f"Day {i+1}: Pred={y_pred[i]:.2f}, Actual={y_true[i]:.2f}")


In [ ]:
#RMSE
mse = mean_squared_error(y_true, pred)
rmse = (mse)**(1/2)
#MAE
mae = mean_absolute_error(y_true, pred)
#MAPE
mape = np.mean(np.abs((y_true - pred) / y_true)) * 100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(range(len(y_true)), y_true, label="Actual Price", color="black")
plt.plot(range(len(y_pred)), y_pred, label="LSTM Forecast (Multivariate)", color="red", linestyle="--")

plt.title("LSTM Forecast vs Actual (AAPL Closing Price with SP500 Exogenous)")
plt.xlabel("Days Ahead (Test Horizon)")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

##### Goldman Sachs

In [ ]:
data = goldman_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

###### Just the close price

In [ ]:
def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])  # predict next 30
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)

#reshape for LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))

#build model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)  # directly predict 30 steps
])

model.compile(optimizer='adam', loss='mse')

#train
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)

#forecast
last_seq = scaled_data[-60:]  # last 60 days
last_seq = last_seq.reshape((1, 60, 1))
pred = model.predict(last_seq)
pred = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()

print("Next 30-day forecast:", pred)

In [ ]:
y_true = data[-30:].flatten()  # these are already in real price units

#compare
for i in range(5):
    print(f"Day {i+1}: Pred={pred[i]:.2f}, Actual={y_true[i]:.2f}")


#RMSE
mse = mean_squared_error(y_true, pred)
rmse = (mse)**(1/2)
#MAE
mae = mean_absolute_error(y_true, pred)
#MAPE
mape = np.mean(np.abs((y_true - pred) / y_true)) * 100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

###### Exog and Close

In [ ]:
goldman_lstm = pd.concat([goldman_df["Close"], sp_500["Close"]], axis=1)
goldman_lstm.columns = ["Goldman_Close", "SP500_Close"]
goldman_lstm = goldman_lstm.dropna()

goldman_lstm.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(goldman_lstm)

In [ ]:
def create_sequences_multivariate(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        #input includes both features
        X.append(data[i-past_steps:i, :])
        #target is only the stock Close (column 0)
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences_multivariate(scaled_data, 60, 30)

In [ ]:
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)  # predict next 30 days
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)

In [ ]:
#last 60 days of features (stock + sp500)
last_seq = scaled_data[-60:]
last_seq = last_seq.reshape((1, 60, 2))

y_pred = model.predict(last_seq)
y_pred = scaler.inverse_transform(
    np.concatenate([y_pred.reshape(-1,1), np.zeros((30,1))], axis=1)
)[:,0]  # keep only stock column

#actual values for comparison
y_true = goldman_lstm['Goldman_Close'].values[-30:]

for i in range(5):
    print(f"Day {i+1}: Pred={y_pred[i]:.2f}, Actual={y_true[i]:.2f}")

In [ ]:
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
rmse = (mse)**(1/2)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

##### Google

In [ ]:
data = google_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])  # predict next 30
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)

#reshape for LSTM
X = X.reshape((X.shape[0], X.shape[1], 1))

#build model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)  # directly predict 30 steps
])

model.compile(optimizer='adam', loss='mse')

#train
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)

#forecast
last_seq = scaled_data[-60:]  # last 60 days
last_seq = last_seq.reshape((1, 60, 1))
pred = model.predict(last_seq)
pred = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()

print("Next 30-day forecast:", pred)

In [ ]:
y_true = data[-30:].flatten()  # these are already in real price units

#compare
for i in range(5):
    print(f"Day {i+1}: Pred={pred[i]:.2f}, Actual={y_true[i]:.2f}")



mse = mean_squared_error(y_true, pred)
rmse = (mse)**(1/2)
mae = mean_absolute_error(y_true, pred)
mape = np.mean(np.abs((y_true - pred) / y_true)) * 100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
#close and exog

google_lstm = pd.concat([google_df["Close"], sp_500["Close"]], axis=1)
google_lstm.columns = ["Google_Close", "SP500_Close"]
google_lstm = google_lstm.dropna()

google_lstm.head()

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(google_lstm)

def create_sequences_multivariate(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        #input includes both features
        X.append(data[i-past_steps:i, :])
        # target is only the stock Close (column 0)
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences_multivariate(scaled_data, 60, 30)

model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)  # predict next 30 days
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)

#last 60 days of features (stock + sp500)
last_seq = scaled_data[-60:]
last_seq = last_seq.reshape((1, 60, 2))

y_pred = model.predict(last_seq)
y_pred = scaler.inverse_transform(
    np.concatenate([y_pred.reshape(-1,1), np.zeros((30,1))], axis=1)
)[:,0]  # keep only stock column

#actual values for comparison
y_true = google_lstm['Google_Close'].values[-30:]

for i in range(5):
    print(f"Day {i+1}: Pred={y_pred[i]:.2f}, Actual={y_true[i]:.2f}")

mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
rmse = (mse)**(1/2)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

##### Citi

In [ ]:
data = citi_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])  # predict next 30
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)


X = X.reshape((X.shape[0], X.shape[1], 1))


model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)
])

model.compile(optimizer='adam', loss='mse')


model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)


last_seq = scaled_data[-60:]  # last 60 days
last_seq = last_seq.reshape((1, 60, 1))
pred = model.predict(last_seq)
pred = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()

print("Next 30-day forecast:", pred)

In [ ]:
y_true = data[-30:].flatten()  # these are already in real price units


for i in range(5):
    print(f"Day {i+1}: Pred={pred[i]:.2f}, Actual={y_true[i]:.2f}")



mse = mean_squared_error(y_true, pred)
rmse = (mse)**(1/2)
mae = mean_absolute_error(y_true, pred)
mape = np.mean(np.abs((y_true - pred) / y_true)) * 100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
#close and exog variable

citi_lstm = pd.concat([citi_df["Close"], sp_500["Close"]], axis=1)
citi_lstm.columns = ["Citi_Close", "SP500_Close"]
citi_lstm = citi_lstm.dropna()

citi_lstm.head()

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(citi_lstm)

def create_sequences_multivariate(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):

        X.append(data[i-past_steps:i, :])

        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences_multivariate(scaled_data, 60, 30)


model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)


last_seq = scaled_data[-60:]
last_seq = last_seq.reshape((1, 60, 2))

y_pred = model.predict(last_seq)
y_pred = scaler.inverse_transform(
    np.concatenate([y_pred.reshape(-1,1), np.zeros((30,1))], axis=1)
)[:,0]


y_true = citi_lstm['Citi_Close'].values[-30:]

for i in range(5):
    print(f"Day {i+1}: Pred={y_pred[i]:.2f}, Actual={y_true[i]:.2f}")

mse = mean_squared_error(y_true, y_pred)

mae = mean_absolute_error(y_true, y_pred)

mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
rmse = (mse)**(1/2)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

##### Johnson and Johnson

In [ ]:
data = johnson_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)


X = X.reshape((X.shape[0], X.shape[1], 1))


model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)
])

model.compile(optimizer='adam', loss='mse')


model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)


last_seq = scaled_data[-60:]
last_seq = last_seq.reshape((1, 60, 1))
pred = model.predict(last_seq)
pred = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()

print("Next 30-day forecast:", pred)

In [ ]:
y_true = data[-30:].flatten()


for i in range(5):
    print(f"Day {i+1}: Pred={pred[i]:.2f}, Actual={y_true[i]:.2f}")



mse = mean_squared_error(y_true, pred)
rmse = (mse)**(1/2)
mae = mean_absolute_error(y_true, pred)
mape = np.mean(np.abs((y_true - pred) / y_true)) * 100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
#close and exog variable

johnson_lstm = pd.concat([johnson_df["Close"], sp_500["Close"]], axis=1)
johnson_lstm.columns = ["Johnson_Close", "SP500_Close"]
johnson_lstm = johnson_lstm.dropna()

johnson_lstm.head()

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(johnson_lstm)

def create_sequences_multivariate(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):

        X.append(data[i-past_steps:i, :])

        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences_multivariate(scaled_data, 60, 30)

model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(30)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.1)


last_seq = scaled_data[-60:]
last_seq = last_seq.reshape((1, 60, 2))

y_pred = model.predict(last_seq)
y_pred = scaler.inverse_transform(
    np.concatenate([y_pred.reshape(-1,1), np.zeros((30,1))], axis=1)
)[:,0]


y_true = johnson_lstm['Johnson_Close'].values[-30:]

for i in range(5):
    print(f"Day {i+1}: Pred={y_pred[i]:.2f}, Actual={y_true[i]:.2f}")

mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
rmse = (mse)**(1/2)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

#### Transformer

##### Apple

In [ ]:
data = apple_df[['Close']].values

#scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

#creating the sequence
def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)

#class to use torch
class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_size = int(len(X)*0.9)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

train_dataset = StockDataset(X_train, y_train)
val_dataset = StockDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
#transformer model
class TransformerForecast(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=3, dropout=0.1, out_len=30):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=128,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, out_len)

    def forward(self, x):
        x = self.input_fc(x)                # [batch, seq_len, d_model]
        x = self.transformer_encoder(x)     # [batch, seq_len, d_model]
        x = x[:, -1, :]                     # use last time step
        out = self.fc_out(x)                # [batch, out_len]
        return out

#training set up
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerForecast().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*xb.size(0)
    train_loss /= len(train_loader.dataset)

    #validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            val_loss += loss.item()*xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

In [ ]:
#forecast the next 30 days
model.eval()
last_seq = torch.tensor(scaled_data[-60:], dtype=torch.float32).unsqueeze(0).to(device)
with torch.no_grad():
    pred = model(last_seq).cpu().numpy()

pred_prices = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
print("Next 30-day forecast:", pred_prices)

#evaluation
y_true = data[-30:].flatten()
mse = mean_squared_error(y_true, pred_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, pred_prices)
mape = np.mean(np.abs((y_true - pred_prices)/y_true))*100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
y_true = data[-30:].flatten()  # these are already in real price units

#compare
for i in range(10):
    print(f"Day {i+1}: Pred={pred_prices[i]:.2f}, Actual={y_true[i]:.2f}")

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(range(len(y_true)), y_true, label="Actual Price", color="black")
plt.plot(range(len(pred_prices)), pred_prices, label="Transformer Forecast", color="red", linestyle="--")

plt.title("Transformer Forecast vs Actual (AAPL Closing Price)")
plt.xlabel("Days Ahead (Test Horizon)")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

##### Citi

In [ ]:
data = citi_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)

class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_size = int(len(X)*0.9)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

train_dataset = StockDataset(X_train, y_train)
val_dataset = StockDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
class TransformerForecast(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=3, dropout=0.1, out_len=30):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=128,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, out_len)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]
        out = self.fc_out(x)
        return out


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerForecast().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


epochs = 20
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*xb.size(0)
    train_loss /= len(train_loader.dataset)


    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            val_loss += loss.item()*xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

In [ ]:
model.eval()
last_seq = torch.tensor(scaled_data[-60:], dtype=torch.float32).unsqueeze(0).to(device)
with torch.no_grad():
    pred = model(last_seq).cpu().numpy()

pred_prices = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
print("Next 30-day forecast:", pred_prices)

y_true = data[-30:].flatten()
mse = mean_squared_error(y_true, pred_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, pred_prices)
mape = np.mean(np.abs((y_true - pred_prices)/y_true))*100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
y_true = data[-30:].flatten()

for i in range(10):
    print(f"Day {i+1}: Pred={pred_prices[i]:.2f}, Actual={y_true[i]:.2f}")

##### Google

In [ ]:
data = google_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)

class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_size = int(len(X)*0.9)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

train_dataset = StockDataset(X_train, y_train)
val_dataset = StockDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
class TransformerForecast(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=3, dropout=0.1, out_len=30):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=128,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, out_len)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]
        out = self.fc_out(x)
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerForecast().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 20
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*xb.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            val_loss += loss.item()*xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

In [ ]:
model.eval()
last_seq = torch.tensor(scaled_data[-60:], dtype=torch.float32).unsqueeze(0).to(device)
with torch.no_grad():
    pred = model(last_seq).cpu().numpy()

pred_prices = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
print("Next 30-day forecast:", pred_prices)

y_true = data[-30:].flatten()
mse = mean_squared_error(y_true, pred_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, pred_prices)
mape = np.mean(np.abs((y_true - pred_prices)/y_true))*100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
y_true = data[-30:].flatten()
for i in range(10):
    print(f"Day {i+1}: Pred={pred_prices[i]:.2f}, Actual={y_true[i]:.2f}")

##### Goldman Sachs

In [ ]:
data = goldman_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)

class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_size = int(len(X)*0.9)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

train_dataset = StockDataset(X_train, y_train)
val_dataset = StockDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
class TransformerForecast(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=3, dropout=0.1, out_len=30):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=128,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, out_len)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]
        out = self.fc_out(x)
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerForecast().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 20
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*xb.size(0)
    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            val_loss += loss.item()*xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

In [ ]:
model.eval()
last_seq = torch.tensor(scaled_data[-60:], dtype=torch.float32).unsqueeze(0).to(device)
with torch.no_grad():
    pred = model(last_seq).cpu().numpy()

pred_prices = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
print("Next 30-day forecast:", pred_prices)


y_true = data[-30:].flatten()
mse = mean_squared_error(y_true, pred_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, pred_prices)
mape = np.mean(np.abs((y_true - pred_prices)/y_true))*100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
y_true = data[-30:].flatten()

for i in range(10):
    print(f"Day {i+1}: Pred={pred_prices[i]:.2f}, Actual={y_true[i]:.2f}")

##### Johnson and Johnson

In [ ]:
data = johnson_df[['Close']].values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

def create_sequences(data, past_steps=60, future_steps=30):
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i-past_steps:i, 0])
        y.append(data[i:i+future_steps, 0])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_data, 60, 30)

class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_size = int(len(X)*0.9)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:], y[train_size:]

train_dataset = StockDataset(X_train, y_train)
val_dataset = StockDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
class TransformerForecast(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=3, dropout=0.1, out_len=30):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=128,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, out_len)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]
        out = self.fc_out(x)
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerForecast().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 20
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*xb.size(0)
    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            val_loss += loss.item()*xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

In [ ]:
model.eval()
last_seq = torch.tensor(scaled_data[-60:], dtype=torch.float32).unsqueeze(0).to(device)
with torch.no_grad():
    pred = model(last_seq).cpu().numpy()

pred_prices = scaler.inverse_transform(pred.reshape(-1, 1)).flatten()
print("Next 30-day forecast:", pred_prices)

y_true = data[-30:].flatten()
mse = mean_squared_error(y_true, pred_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, pred_prices)
mape = np.mean(np.abs((y_true - pred_prices)/y_true))*100

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
y_true = data[-30:].flatten()

for i in range(10):
    print(f"Day {i+1}: Pred={pred_prices[i]:.2f}, Actual={y_true[i]:.2f}")